In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
#import yaml
import requests
import json
from pandas.io.json import json_normalize

**1. QS World Ranking website scrapping**

In [2]:
url = "https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508196855735"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [4]:
results_page = BeautifulSoup(response.content.decode("utf-8", errors='ignore'),'html.parser')
data_string=str(results_page)


In [5]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
l=cleanhtml(str(data_string))
m=json.dumps(l)
n=json.loads(m )

#json_acceptable_string = n.replace("'", "\"")
#r = json.loads(json_acceptable_string)
n

'{"data":[{"nid":"294850","url":"\\/universities\\/massachusetts-institute-technology-mit","title":"Massachusetts Institute of Technology (MIT)","logo":"\\u003Cimg src=\\u0022https:\\/\\/www.topuniversities.com\\/sites\\/default\\/files\\/massachusetts-institute-of-technology-mit_410_small_0.jpg\\u0022 alt=\\u0022Massachusetts Institute of Technology (MIT)  Logo\\u0022\\u003E","core_id":"410","score":"100","rank_display":"1","country":"United States","cc":"US","region":"North America","stars":"6","guide":"\\u003Ca href=\\u0022\\/where-to-study\\/north-america\\/united-states\\/guide\\u0022 class=\\u0022guide-link\\u0022 target=\\u0022_blank\\u0022\\u003EUnited States\\u003C\\/a\\u003E"},{"nid":"297282","url":"\\/universities\\/stanford-university","title":"Stanford University","logo":"\\u003Cimg src=\\u0022https:\\/\\/www.topuniversities.com\\/sites\\/default\\/files\\/stanford-university_573_small_0.jpg\\u0022 alt=\\u0022Stanford University Logo\\u0022\\u003E","core_id":"573","score":

In [7]:
def extractNumber(div):
    return div.find('div',class_="number").text.strip()
k=json.loads(n)
df = pd.DataFrame.from_dict(json_normalize(k["data"][:]), orient='columns')
df=df.drop(['guide','logo','core_id','nid','cc','stars'],axis=1)
df=pd.concat([df,pd.DataFrame(columns=['totalStaff','internationalStaff','totalStudents','internationalStudents'])])
baseurl='https://www.topuniversities.com'
#testcounter=0
for index,path in enumerate(df['url']):
    
    if index>200:
        df
        break
    #Retrieve html
    response = requests.get(baseurl+path)
    if response.status_code == 200:
        #Create a html parser
        soup=BeautifulSoup(response.content,'html.parser')
        try:
            #find the class total faculty in which we can extract the usefull number.
            mydivs = soup.find_all("div", { "class" : "total faculty" })
            df['totalStaff'].iloc[index]=extractNumber(mydivs[0])
        except IndexError:
            print('missing total staff:  '+str(baseurl+path))
        
        
        
        try:
            mydivs = soup.find_all("div", { "class" : "inter faculty" })
            df['internationalStaff'].iloc[index]=extractNumber(mydivs[0])
        except IndexError:
            print('missing international staff '+str(baseurl+path))

        try:
            mydivs=soup.find_all("div", { "class" : "students-main wrapper col-md-4" })
            df['totalStudents'].iloc[index]=extractNumber(mydivs[0])
        except IndexError:
            print('missing total students:   '+str(baseurl+path))



        try:
            mydivs=soup.find_all("div", { "class" : "int-students-main wrapper col-md-4" })
            df['internationalStudents'].iloc[index]=extractNumber(mydivs[0])
        except IndexError:
            print('missing international students: '+str(baseurl+path))
        
            

        
    else:
        print("Failure")

missing total staff:  https://www.topuniversities.com/universities/new-york-university-nyu
missing international staff https://www.topuniversities.com/universities/new-york-university-nyu
missing total students:   https://www.topuniversities.com/universities/new-york-university-nyu
missing international students: https://www.topuniversities.com/universities/new-york-university-nyu
missing international staff https://www.topuniversities.com/universities/indian-institute-science-iisc-bangalore


In [12]:
df

country internationalStaff internationalStudents rank_display  \
0     United States              1,679                 3,717            1   
1     United States              2,042                 3,611            2   
2     United States              1,311                 5,266            3   
3     United States                350                   647            4   
4    United Kingdom              2,278                 6,699            5   
5    United Kingdom              2,964                 7,353            6   
6    United Kingdom              2,554                14,854            7   
7    United Kingdom              2,071                 8,746            8   
8     United States                635                 3,379            9   
9       Switzerland              1,886                 7,563           10   
10        Singapore              2,993                 7,251           11   
11      Switzerland              1,300                 5,896           12   
12    United States                246                 1,793           13   
13    United States                970                 5,411           14   
14        Singapore              3,086                 8,917           15   
15    United States              1,708                 2,469           16   
16    United States              1,061                 4,105           17   
17    United States                913                 8,105           18   
18    United States              1,383                 4,250           19   
19        Australia                927                 5,551           20   
20    United States              1,920                 7,527          =21   
21    United States                226                 2,864          =21   
22   United Kingdom              1,553                10,551          =23   
23   United Kingdom              1,513                 9,451          =23   
24            China                932                 4,072           25   
25        Hong Kong              2,085                 8,230           26   
26    United States              1,395                 6,086           27   
27            Japan                254                 2,639          =28   
28    United States              1,141                 3,422          =28   
29        Hong Kong                835                 2,921           30   
..              ...                ...                   ...          ...   
929          Serbia                NaN                   NaN     801-1000   
930       Indonesia                NaN                   NaN     801-1000   
931  United Kingdom                NaN                   NaN     801-1000   
932  United Kingdom                NaN                   NaN     801-1000   
933        Pakistan                NaN                   NaN     801-1000   
934           Ghana                NaN                   NaN     801-1000   
935  United Kingdom                NaN                   NaN     801-1000   
936        Pakistan                NaN                   NaN     801-1000   
937        Pakistan                NaN                   NaN     801-1000   
938        Slovenia                NaN                   NaN     801-1000   
939   United States                NaN                   NaN     801-1000   
940   United States                NaN                   NaN     801-1000   
941           India                NaN                   NaN     801-1000   
942           Kenya                NaN                   NaN     801-1000   
943       Australia                NaN                   NaN     801-1000   
944           India                NaN                   NaN     801-1000   
945   United States                NaN                   NaN     801-1000   
946   United States                NaN                   NaN     801-1000   
947     Philippines                NaN                   NaN     801-1000   
948   United States                NaN                   NaN     801-1000   
949     

In [33]:
reserveDf=df.copy()

**2. Times Higher Education website scrapping**

In [2]:
url = "https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [3]:
results_page = BeautifulSoup(response.content.decode("utf-8", errors='ignore'),'html.parser')
data_string=str(results_page)

In [4]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext
l=cleanhtml(str(data_string))
m=json.dumps(l)
n=json.loads(m )

#json_acceptable_string = n.replace("'", "\"")
#r = json.loads(json_acceptable_string)
n

'{"data":[{"rank_order":"10","rank":"1","name":"University of Oxford","scores_overall":"94.3","scores_overall_rank":"10","scores_teaching":"86.7","scores_teaching_rank":"5","scores_research":"99.5","scores_research_rank":"1","scores_citations":"99.1","scores_citations_rank":"15","scores_industry_income":"63.7","scores_industry_income_rank":"169","scores_international_outlook":"95.0","scores_international_outlook_rank":"24","record_type":"master_account","member_level":"0","url":"\\/world-university-rankings\\/university-oxford","nid":468,"location":"United Kingdom","stats_number_students":"20,409","stats_student_staff_ratio":"11.2","stats_pc_intl_students":"38%","stats_female_male_ratio":"46 : 54","aliases":"University of Oxford","subjects_offered":"Archaeology,Art, Performing Arts \\u0026 Design,Biological Sciences,Business \\u0026 Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics \\u0026 Econometrics,Electrical \\u0026 Electronic Engineering,Gener

In [5]:
#Output to DataFrame
k=json.loads(n)
df2 = pd.DataFrame.from_dict(json_normalize(k["data"][:]), orient='columns')
df2

aliases            location  \
0                                  University of Oxford      United Kingdom   
1                               University of Cambridge      United Kingdom   
2            California Institute of Technology caltech       United States   
3                                   Stanford University       United States   
4                 Massachusetts Institute of Technology       United States   
5                                    Harvard University       United States   
6                                  Princeton University       United States   
7                               Imperial College London      United Kingdom   
8                                 University of Chicago       United States   
9     ETH Zurich – Swiss Federal Institute of Techno...         Switzerland   
10                           University of Pennsylvania       United States   
11                                      Yale University       United States   
12                             Johns Hopkins University       United States   
13                                  Columbia University       United States   
14                University of California, Los Angeles       United States   
15                        University College London ucl      United Kingdom   
16                                      Duke University       United States   
17                   University of California, Berkeley       United States   
18                                   Cornell University       United States   
19                              Northwestern University       United States   
20                               University of Michigan       United States   
21                 National University of Singapore nus           Singapore   
22                                University of Toronto              Canada   
23                           Carnegie Mellon University       United States   
24     London School of Economics and Political Science      United Kingdom   
25                             University of Washington       United States   
26                              University of Edinburgh      United Kingdom   
27                                  New York University       United States   
28                                    Peking University               China   
29    Tsinghua University Qinghua University Tsing h...               China   
...                                                 ...                 ...   
1072                              Sathyabama University               India   
1073                                Sechenov University  Russian Federation   
1074                       Shanghai Maritime University               China   
1075             Shibaura Institute of Technology Tokyo               Japan   
1076                        Siberian Federal University  Russian Federation   
1077               Sidi Mohamed Ben Abdellah University             Morocco   
1078                     University of the Sinos Valley              Brazil   
1079                                   Sofia University            Bulgaria   
1080                      Southwest Jiaotong University               China   
1081                                     SRM University               India   
1082                        State University of Maringá              Brazil   
1083      State University of Western Paraná (Unioeste)              Brazil   
1084                 Tamil Nadu Agricultural University               India   
1085                                 Tamkang University              Taiwan   
1086                    Technical University of Liberec      Czech Republic   
1087                              Tokyo City University               Japan   
1088                             Tokyo Denki University               Japan   
1089                                 Tottori University               Japan   
1090                                    Toyo University               Japan   
1091               

In [27]:
#Cleaning up the DataFrame
dftwo = df2[['location','name','rank','stats_number_students','stats_student_staff_ratio','stats_pc_intl_students']].copy()
#dftwo['rank']
i = 0

bla = 0
dftwo['rank']=dftwo['rank'].astype(str)

for index,f in enumerate(dftwo['rank']):
    f=f.replace("=","")
    f=f.split("–")[0]
    if int(f)>200: 
        indexFinal=index
        break

dftwo=dftwo.iloc[0:indexFinal]

**3. Merging Data Frames**

**4. Exploratory**

**5. Best University**